In [ ]:
import os
import re
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification


# ---------------------- 1. 核心配置（请根据实际情况修改） ----------------------
class FastConfig:
    def __init__(self):
        # 1. 已训练模型/词典路径（训练结果文件夹）
        self.TRAINED_FOLDER = "financial_bert_results"
        self.MODEL_WEIGHT = os.path.join(self.TRAINED_FOLDER, "financial_bert_best_model.pth")
        self.SENTIMENT_DICT = os.path.join(self.TRAINED_FOLDER, "financial_bert_sentiment_dict.json")
        
        # 2. 本地BERT基础模型路径（与训练时一致）
        self.LOCAL_BERT = r"hf_cach"
        
        # 3. 输入数据路径
        self.BAR_DATA = "玉米_股吧数据_2020-20244.xlsx"  # 股吧数据
        self.STOCK_DATA = "玉米.csv"  # 股票数据
        self.STOPWORDS = "stopwords.txt"  # 停用词表
        
        # 4. 输出路径
        self.OUTPUT = "玉米_带情感得分.csv"
        
        # 5. 模型参数
        self.BATCH_SIZE = 16  # CPU建议8，GPU建议16-32
        self.MAX_LEN = 128    # 文本最大长度
        self.DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # 6. 日期列配置 - 请根据你的股票数据修改！
        self.STOCK_DATE_COLUMN = "date"  # 股票数据中的日期列名称
        self.BAR_DATE_COLUMN = "最后更新时间"  # 股吧数据中的日期列名称


# ---------------------- 2. 工具函数 ----------------------
def load_stopwords(stop_path):
    """加载停用词表"""
    try:
        with open(stop_path, 'r', encoding='utf-8') as f:
            return set(f.read().splitlines())
    except Exception as e:
        print(f"⚠️ 停用词表加载失败: {str(e)}，将跳过停用词过滤")
        return set()


def load_sentiment_dict(dict_path):
    """加载情感词典"""
    try:
        with open(dict_path, 'r', encoding='utf-8') as f:
            raw = json.load(f)
        return {w: (raw[w]["sentiment"], raw[w]["weight"]) for w in raw}
    except Exception as e:
        raise ValueError(f"❌ 情感词典加载失败: {str(e)}")


def clean_text(text, stopwords, sent_dict):
    """清洗文本"""
    text = str(text).strip()
    if not text:
        return ""
    # 保留关键字符
    text = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fa5\s%￥.\+\-!?]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    # 过滤停用词（保留情感词典中的词）
    if stopwords:
        words = [w for w in text.split() if w not in stopwords or w in sent_dict]
        return " ".join(words) if words else text
    return text


# ---------------------- 3. 数据集定义 ----------------------
class FastBarDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df['cleaned_text'].tolist()
        self.titles = df['标题'].tolist()
        self.times = df[FastConfig().BAR_DATE_COLUMN].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'title': self.titles[idx],
            'time': self.times[idx]
        }


# ---------------------- 4. 日期处理工具函数 ----------------------
def parse_date_column(df, date_column, column_name):
    """解析日期列，支持多种格式"""
    # 尝试多种日期格式解析
    date_formats = [
        '%Y-%m-%d', '%Y/%m/%d', '%Y年%m月%d日',
        '%d-%m-%Y', '%d/%m/%Y', '%m-%d-%Y', '%m/%d/%Y',
        '%Y-%m-%d %H:%M:%S', '%Y/%m/%d %H:%M:%S'
    ]
    
    for fmt in date_formats:
        try:
            parsed = pd.to_datetime(df[date_column], format=fmt, errors='coerce')
            if parsed.notna().sum() > 0.5 * len(df):  # 至少50%的数据能被解析
                return parsed.dt.date.astype(str)
        except:
            continue
    
    # 最后尝试自动解析
    parsed = pd.to_datetime(df[date_column], errors='coerce')
    valid_rate = parsed.notna().sum() / len(df)
    
    if valid_rate < 0.1:  # 有效解析率低于10%
        raise ValueError(f"""
        ❌ 日期解析失败！
        无法解析{column_name}中的日期格式，请检查：
        1. 列名是否正确：当前使用 '{date_column}'
        2. 日期格式是否符合常见格式（如2023-01-01、2023/01/01）
        3. 数据中是否包含非日期值
        """)
    
    print(f"⚠️ 部分日期解析失败（有效率：{valid_rate:.2%}），已自动处理")
    return parsed.dt.date.astype(str)


# ---------------------- 5. 核心流程 ----------------------
def fast_predict_and_merge():
    config = FastConfig()
    print(f"✅ 开始：情感预测与数据合并（设备：{config.DEVICE.type}）")

    # 步骤1：加载模型
    print("\n1/3 加载模型...")
    try:
        tokenizer = BertTokenizer.from_pretrained(
            config.LOCAL_BERT, local_files_only=True, do_lower_case=False
        )
        model = BertForSequenceClassification.from_pretrained(
            config.LOCAL_BERT, num_labels=3, local_files_only=True
        )
        model.load_state_dict(torch.load(config.MODEL_WEIGHT, map_location=config.DEVICE))
        model.to(config.DEVICE).eval()
    except Exception as e:
        raise ValueError(f"❌ 模型加载失败: {str(e)}\n请检查模型路径是否正确")

    # 步骤2：处理股吧数据并预测情感
    print("\n2/3 处理股吧数据并预测情感...")
    try:
        # 加载股吧数据
        if Path(config.BAR_DATA).suffix == '.xlsx':
            bar_df = pd.read_excel(config.BAR_DATA)
        else:
            bar_df = pd.read_csv(config.BAR_DATA, encoding='utf-8-sig')
        
        # 检查必要字段
        required_bar_cols = ['标题', config.BAR_DATE_COLUMN]
        missing_bar_cols = [col for col in required_bar_cols if col not in bar_df.columns]
        if missing_bar_cols:
            raise ValueError(f"❌ 股吧数据缺少字段：{missing_bar_cols}")
        
        # 数据清洗
        bar_df = bar_df[required_bar_cols].dropna().drop_duplicates('标题').reset_index(drop=True)
        print(f"  股吧数据清洗后：{len(bar_df)}条有效记录")
        
        # 文本预处理
        stopwords = load_stopwords(config.STOPWORDS)
        sent_dict = load_sentiment_dict(config.SENTIMENT_DICT)
        bar_df['cleaned_text'] = bar_df['标题'].apply(lambda x: clean_text(x, stopwords, sent_dict))
        bar_df = bar_df[bar_df['cleaned_text'].str.len() >= 3].reset_index(drop=True)
        print(f"  文本预处理后：{len(bar_df)}条有效记录")

        # 批量预测情感得分
        dataset = FastBarDataset(bar_df, tokenizer, config.MAX_LEN)
        dataloader = DataLoader(dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0)
        
        all_scores = []
        all_titles = []
        all_times = []
        with torch.no_grad():  # 关闭梯度计算，加速
            for batch in tqdm(dataloader, desc="预测进度"):
                input_ids = batch['input_ids'].to(config.DEVICE)
                attention_mask = batch['attention_mask'].to(config.DEVICE)
                logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
                probs = torch.softmax(logits, dim=1).cpu().numpy()
                scores = probs[:, 2] - probs[:, 0]  # 积极概率 - 消极概率
                all_scores.extend(scores)
                all_titles.extend(batch['title'])
                all_times.extend(batch['time'])

        # 整理情感得分结果
        score_df = pd.DataFrame({
            '标题': all_titles,
            config.BAR_DATE_COLUMN: all_times,
            '情感得分': [round(s, 4) for s in all_scores]
        })
        bar_with_score = bar_df.merge(score_df, on=['标题', config.BAR_DATE_COLUMN], how='left')
    except Exception as e:
        raise ValueError(f"❌ 股吧数据处理失败: {str(e)}")

    # 步骤3：合并到股票数据
    print("\n3/3 合并到股票数据...")
    try:
        # 加载股票数据
        stock_df = pd.read_csv(config.STOCK_DATA, encoding='utf-8-sig')
        
        # 显示股票数据列信息，帮助调试
        print(f"  股票数据包含列：{stock_df.columns.tolist()}")
        
        # 检查股票日期列是否存在
        if config.STOCK_DATE_COLUMN not in stock_df.columns:
            raise ValueError(f"❌ 股票数据中未找到日期列 '{config.STOCK_DATE_COLUMN}'\n请修改STOCK_DATE_COLUMN为实际列名")
        
        # 解析日期（核心修复部分）
        print(f"  解析股吧日期列：{config.BAR_DATE_COLUMN}")
        bar_with_score['日期'] = parse_date_column(bar_with_score, config.BAR_DATE_COLUMN, "股吧数据")
        
        print(f"  解析股票日期列：{config.STOCK_DATE_COLUMN}")
        stock_df['日期'] = parse_date_column(stock_df, config.STOCK_DATE_COLUMN, "股票数据")
        
        # 过滤无效日期
        bar_with_score = bar_with_score[bar_with_score['日期'] != 'NaT'].reset_index(drop=True)
        stock_df = stock_df[stock_df['日期'] != 'NaT'].reset_index(drop=True)

        # 计算每日平均情感得分
        daily_score = bar_with_score.groupby('日期')['情感得分'].agg(['mean', 'count']).reset_index()
        daily_score.columns = ['日期', '每日平均情感得分', '当日股吧评论数']
        print(f"  计算完成：{len(daily_score)}个日期的情感得分")

        # 合并数据
        merged_df = stock_df.merge(
            daily_score, on='日期', how='left'
        ).fillna({'每日平均情感得分': 0.0, '当日股吧评论数': 0})

        # 保存结果
        merged_df.to_csv(config.OUTPUT, index=False, encoding='utf-8-sig')
        print(f"\n🎉 全部完成！结果已保存至：{config.OUTPUT}")
        
        # 预览结果
        print("\n结果预览（前3行）：")
        preview_cols = ['日期', '每日平均情感得分', '当日股吧评论数', 'open', 'close']
        preview_cols = [c for c in preview_cols if c in merged_df.columns]
        print(merged_df[preview_cols].head(3))
        
    except Exception as e:
        raise ValueError(f"❌ 股票数据合并失败: {str(e)}")


# ---------------------- 运行入口 ----------------------
if __name__ == "__main__":
    try:
        fast_predict_and_merge()
    except Exception as e:
        print(f"\n❌ 执行终止：{str(e)}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 加载数据
df = pd.read_csv('玉米_带情感得分.csv')

# 假设使用每日平均情感得分和当日股吧评论数来预测收盘价
features = ['每日平均情感得分', '当日股吧评论数', 'close']
data = df[features].values

# 归一化数据
sc = MinMaxScaler(feature_range=(0, 1))
scaled_data = sc.fit_transform(data)

# 划分训练集和测试集
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size, :]
test_data = scaled_data[train_size:, :]

# 准备训练数据
def create_dataset(dataset, time_step=1):
    X, y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), :])
        y.append(dataset[i + time_step, 2])
    return np.array(X), np.array(y)

time_step = 10
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# 定义模型类型为多层 LSTM
model_type = 1
n_epochs = 20

# 建构 LSTM 模型
if model_type == 1:
    # 单层 LSTM
    model = Sequential()
    model.add(LSTM(units=50, activation='relu',
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(units=1))
if model_type == 2:
    # 多层 LSTM
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', return_sequences=True,
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50, activation='relu'))
    model.add(Dense(1))
if model_type == 3:
    # 双向 LSTM
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu'),
                            input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# 训练模型
history = model.fit(X_train, y_train, batch_size=64, epochs=n_epochs,
                    validation_data=(X_test, y_test), validation_freq=1)

# 预测
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# 反归一化预测结果
# 构建与原始数据相同结构的数组，用于反归一化
train_predict_extended = np.zeros((len(train_predict), data.shape[1]))
train_predict_extended[:, 2] = train_predict[:, 0]
train_predict = sc.inverse_transform(train_predict_extended)[:, 2]

test_predict_extended = np.zeros((len(test_predict), data.shape[1]))
test_predict_extended[:, 2] = test_predict[:, 0]
test_predict = sc.inverse_transform(test_predict_extended)[:, 2]

# 反归一化真实值
y_train_extended = np.zeros((len(y_train), data.shape[1]))
y_train_extended[:, 2] = y_train
y_train = sc.inverse_transform(y_train_extended)[:, 2]

y_test_extended = np.zeros((len(y_test), data.shape[1]))
y_test_extended[:, 2] = y_test
y_test = sc.inverse_transform(y_test_extended)[:, 2]

# 计算评估指标
mse_train = mean_squared_error(y_train, train_predict)
mse_test = mean_squared_error(y_test, test_predict)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
mae_train = mean_absolute_error(y_train, train_predict)
mae_test = mean_absolute_error(y_test, test_predict)

print('训练集 MSE:', mse_train)
print('测试集 MSE:', mse_test)
print('训练集 RMSE:', rmse_train)
print('测试集 RMSE:', rmse_test)
print('训练集 MAE:', mae_train)
print('测试集 MAE:', mae_test)

# 绘制训练和验证损失图
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss by LSTM')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
import matplotlib.pyplot as plt
# 新增导入MAPE计算函数
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# 加载数据
df = pd.read_csv('玉米_带情感得分.csv')

# 假设使用每日平均情感得分和当日股吧评论数来预测收盘价
features = ['每日平均情感得分', '当日股吧评论数', 'close']
data = df[features].values

# 归一化数据
sc = MinMaxScaler(feature_range=(0, 1))
scaled_data = sc.fit_transform(data)

# 划分训练集和测试集
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size, :]
test_data = scaled_data[train_size:, :]

# 准备训练数据
def create_dataset(dataset, time_step=1):
    X, y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), :])
        y.append(dataset[i + time_step, 2])
    return np.array(X), np.array(y)

time_step = 10
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# 定义模型类型为多层 LSTM
model_type = 2
n_epochs = 10

# 建构 LSTM 模型
if model_type == 1:
    # 单层 LSTM
    model = Sequential()
    model.add(LSTM(units=50, activation='relu',
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(units=1))
if model_type == 2:
    # 多层 LSTM
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', return_sequences=True,
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50, activation='relu'))
    model.add(Dense(1))
if model_type == 3:
    # 双向 LSTM
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu'),
                            input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# 训练模型
history = model.fit(X_train, y_train, batch_size=64, epochs=n_epochs,
                    validation_data=(X_test, y_test), validation_freq=1)

# 预测
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# 反归一化预测结果
# 构建与原始数据相同结构的数组，用于反归一化
train_predict_extended = np.zeros((len(train_predict), data.shape[1]))
train_predict_extended[:, 2] = train_predict[:, 0]
train_predict = sc.inverse_transform(train_predict_extended)[:, 2]

test_predict_extended = np.zeros((len(test_predict), data.shape[1]))
test_predict_extended[:, 2] = test_predict[:, 0]
test_predict = sc.inverse_transform(test_predict_extended)[:, 2]

# 反归一化真实值
y_train_extended = np.zeros((len(y_train), data.shape[1]))
y_train_extended[:, 2] = y_train
y_train = sc.inverse_transform(y_train_extended)[:, 2]

y_test_extended = np.zeros((len(y_test), data.shape[1]))
y_test_extended[:, 2] = y_test
y_test = sc.inverse_transform(y_test_extended)[:, 2]

# 计算评估指标（新增MAPE计算）
mse_train = mean_squared_error(y_train, train_predict)
mse_test = mean_squared_error(y_test, test_predict)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
mae_train = mean_absolute_error(y_train, train_predict)
mae_test = mean_absolute_error(y_test, test_predict)
# 新增MAPE计算（乘以100转为百分比形式，更直观）
mape_train = mean_absolute_percentage_error(y_train, train_predict) * 100
mape_test = mean_absolute_percentage_error(y_test, test_predict) * 100

# 打印指标（新增MAPE输出）
print('训练集 MSE:', mse_train)
print('测试集 MSE:', mse_test)
print('训练集 RMSE:', rmse_train)
print('测试集 RMSE:', rmse_test)
print('训练集 MAE:', mae_train)
print('测试集 MAE:', mae_test)
print('训练集 MAPE (%):', mape_train)  # 新增MAPE打印
print('测试集 MAPE (%):', mape_test)  # 新增MAPE打印

# 绘制训练和验证损失图
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss by LSTM')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()